We are considering a small dataset of 320 line which classifies a sentence as truthful or deceptive. The pre trained model we have used here is BERT.<break>
BERT is based on the transformer architecture, which was introduced in the paper "Attention is All You Need" by Vaswani et al. The transformer architecture uses self-attention mechanisms to capture contextual information from both left and right contexts.
it has 12 transformer layer and 12 attention heads.

In [1]:
import pandas as pd

In [12]:
from sklearn.model_selection import train_test_split
from collections import Counter
path = '/content/review_dataset_train.txt'
# Read dataset from file
with open(path, 'r') as file:
    lines = file.readlines()

# Extract labels and sentences
data = [line.strip().split(' ', 1) for line in lines]
labels, sentences = zip(*data)

# Count the total sentences and each label
total_sentences = len(sentences)
label_counts = Counter(labels)
print('Total no of lines:',total_sentences)
print('Label count for each:',label_counts)

Total no of lines: 320
Label count for each: Counter({'deceptive': 164, 'truthful': 156})


In [13]:
# Display total counts
print("\nTotal Counts:")
print(f"Total Sentences: {total_sentences}")
for label, count in label_counts.items():
    print(f"{label} Count: {count}")


Total Counts:
Total Sentences: 320
truthful Count: 156
deceptive Count: 164


In [14]:
from sklearn.model_selection import train_test_split


# Read dataset from file
with open(path, 'r') as file:
    lines = file.readlines()

# Extract labels and sentences
data = [line.strip().split(' ', 1) for line in lines]
labels, sentences = zip(*data)

# Split the dataset into training and test sets (80:20 ratio)
train_sentences, test_sentences, train_labels, test_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)

In [23]:
print('Length of Train',len(train_sentences))
print('Length of Test',len(test_sentences))

Length of Train 256
Length of Test 64


In [15]:
# Display the first 5 lines of the training set
print("Training Set:")
for label, sentence in zip(train_labels[:5], train_sentences[:5]):
    print(f"{label}: {sentence}")


Training Set:
truthful: My family booked five nitghts at the Omni Chicago for our upcoming spring break. We took advantage of a pre-paid, non-cancellable offer. We subsequently received a 'Bed & Breakfast' offer via email and immediately called the hotel to inquire whether we could switch our reservation (same dates, different offer). We called the 800' number and the customer services representative was so obnoxiously condescending and ignorant, we simply hung up. We had been looking forward to our vacation, but now I am not sure I made the right choice with this hotel. I should have paid closer attention to the reviews - the Ritz, the Four Seasons, and the Peninsula would have never treated us this way. I will post again after our stay on April 17th.
deceptive: While travelling for business I had my family join me and we stayed at the Fairmont Chicago, Millennium Park. It is breathtakingly beautiful and plush. The rates for rooms, though not the cheapest choice are quite reasonable f

In [16]:
# Display the first 5 lines of the test set
print("\nTest Set:")
for label, sentence in zip(test_labels[:5], test_sentences[:5]):
    print(f"{label}: {sentence}")


Test Set:
deceptive: I stayed at the Hard Rock hotel in Chicago last year and can say i was not satisfied with my experience. I feel that it didn't live up to what was advertised and the accommodations were sub-par at best. I had the chance to catch a show and the music was too loud and the sound guy's work resembled that of an amateur. My personal rating is 2/5 stars and i would not recommend this hotel to a friend.
truthful: My husband and I stayed overnight at the Affinia. Prior reviews led us to believe this was a quality hotel; not true. Since it was a very special occasion, and also because we were only staying overnight, we didn't let the shabbiness and poor service of the hotel ruin our weekend. However, this would have qualified as one of the poorest hotel experiences. I've ever had. First, the smells: The linens and pillows on our bed smelled moldy and mildewy. The odors were not mild, and I found it pretty repulsive. I couldn't wait to jump out of bed in the morning, to get

In [17]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [18]:
from sklearn.preprocessing import LabelEncoder
# Assume train_labels and test_labels are lists of strings
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

# Convert labels to PyTorch tensors
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

In [19]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize the input data
train_encodings = tokenizer(train_sentences, truncation=True, padding=True, max_length=128, return_tensors='pt')
test_encodings = tokenizer(test_sentences, truncation=True, padding=True, max_length=128, return_tensors='pt')

# Convert labels to PyTorch tensors
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Create DataLoader for training and test sets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Set up GPU/CPU and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        inputs = {'input_ids': batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'labels': batch[2].to(device)}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-19-a25db235a46c>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels)
<ipython-input-19-a25db235a46c>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_labels = torch.tensor(test_labels)
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a 

In [20]:
# Evaluation
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        inputs = {'input_ids': batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'labels': None}
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).cpu().numpy()
        test_predictions.extend(predictions)

Testing: 100%|██████████| 8/8 [00:32<00:00,  4.01s/it]


In [21]:
# Calculate accuracy
accuracy = accuracy_score(test_labels.numpy(), test_predictions)
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.84375




```
The Test Accuracy is ~ 85% which is fairly good. Other improvements that can be made to make the model better is
```


```
1.   Increase the dataset size,here we have considered a small dataset of 320 lines
2.   Hyperparameter tuning: Experiment with weights,epochs during training the data
3.   Add regularization and dropouts
4.   We can also experiment with different pre trained models such as gpt,XLNet
5.   Make some changes and experiment with model by freezing few layers and add aditional layers.
6.  Maintain balanced dataset,i.e makesure it isnt skewed


```



